<a href="https://colab.research.google.com/github/Harinadhavasala/Chatbot_Code/blob/main/Copy_of_Data_lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
import pandas as pd

In [183]:
df=pd.read_csv("datalake.csv")

In [184]:
df.head()

,LastName,FirstName,Name,Email
0,As Chakravarthi aaaaa,NaN,As Chakravarthi,aschakrav@gmail.com
1,Raveendra xyz,Meda,Meda Raveendra,.mrreddy@ramcadds.in
2,Kannan,NaN,Kannan,kannan.@ican1891@gmail.com+_=
3,Ramyaa,NaN,Ramyaa,ramyaa.doc___@gmail.com
4,Vasan.K,NaN,Vasan.K,90va_sankm@yahoo.com-+


In [185]:
text_cols = df.select_dtypes(include="object").columns
df[text_cols] = df[text_cols].fillna("")

In [186]:
import re
for col in ['FirstName', 'LastName']:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace(r'[^a-zA-Z ]', '', regex=True)



In [187]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NameValidation") \
    .getOrCreate()


In [188]:
spark_df = spark.createDataFrame(df)


In [189]:
from pyspark.sql import functions as F

spark_df = spark_df.withColumn(
    "full_name",
    F.trim(F.concat_ws(" ", F.col("FirstName"), F.col("LastName")))
)


In [190]:
spark_df.show(truncate=False)



+---------------------+---------------+---------------------------+-------------------------------+-----------------------------+
|LastName             |FirstName      |Name                       |Email                          |full_name                    |
+---------------------+---------------+---------------------------+-------------------------------+-----------------------------+
|As Chakravarthi aaaaa|               |As Chakravarthi            |aschakrav@gmail.com            |As Chakravarthi aaaaa        |
|Raveendra   xyz      |Meda           |Meda Raveendra             |.mrreddy@ramcadds.in           |Meda Raveendra   xyz         |
|Kannan               |               |Kannan                     |kannan.@ican1891@gmail.com+_=  |Kannan                       |
|Ramyaa               |               |Ramyaa                     |ramyaa.doc___@gmail.com        |Ramyaa                       |
|VasanK               |               |Vasan.K                    |90va_sankm@yahoo.com-+ 

In [191]:
# from pyspark.sql import functions as F

# spark_df = spark_df.withColumn(
#     "invalid_category",

#     # 1. Empty or blank
#     F.when(
#         F.col("full_name").rlike(r"^\s*$"),
#         "CATEGORY_EMPTY_NAME"
#     )

#     # 2. Blocking / placeholder words
#     .when(
#         F.col("full_name").rlike(
#             r"(?i)\b(do not call|dont call|no call|remove|xyz|abcd|abc|test|poly|elangovan)\b"
#         ),
#         "Invalid Name"
#     )

#     # 3. Only single-letter / dot initial
#     .when(
#         F.col("full_name").rlike(r"^\s*\.?[A-Za-z]\.?\s*$"),
#         "CATEGORY_ONLY_INITIAL"
#     )
# )


In [192]:
from pyspark.sql import functions as F

spark_df = spark_df.withColumn(
    "invalid_names",

    # 1. Empty or blank
    F.when(
        F.col("full_name").rlike(r"^\s*$"),
        "Category_empty_name"
    )

    # 2. Only dots (., .., ...)
    .when(
        F.col("full_name").rlike(r"^\s*\.{1,}\s*$"),
        "Category_only_dots"
    )

    # 3. Only single initial (R, R., .R)
    .when(
        F.col("full_name").rlike(r"^\s*\.?[A-Za-z]\.?\s*$"),
        "Category_Only_Initial"
    )

    # 4. Blocking / placeholder words
    .when(
        F.col("full_name").rlike(
            r"(?i)\b(do not call|dont call|no call|remove|xyz|abcd|abc|test|poly|elangovan)\b"
        ),
        "  Category_Invalid_Name"
    )
)


In [193]:
spark_df = spark_df.withColumn(
    "is_valid",
    F.when(F.col("invalid_names").isNotNull(), False)
     .otherwise(True)
)


In [194]:
spark_df.show(truncate=False)


+---------------------+---------------+---------------------------+-------------------------------+-----------------------------+-----------------------+--------+
|LastName             |FirstName      |Name                       |Email                          |full_name                    |invalid_names          |is_valid|
+---------------------+---------------+---------------------------+-------------------------------+-----------------------------+-----------------------+--------+
|As Chakravarthi aaaaa|               |As Chakravarthi            |aschakrav@gmail.com            |As Chakravarthi aaaaa        |NULL                   |true    |
|Raveendra   xyz      |Meda           |Meda Raveendra             |.mrreddy@ramcadds.in           |Meda Raveendra   xyz         |  Category_Invalid_Name|false   |
|Kannan               |               |Kannan                     |kannan.@ican1891@gmail.com+_=  |Kannan                       |NULL                   |true    |
|Ramyaa               

In [195]:
from pyspark.sql import functions as F

spark_df = (
    spark_df
    # convert FirstName and LastName to lowercase
    .withColumn("FirstName", F.lower(F.col("FirstName")))
    .withColumn("LastName", F.lower(F.col("LastName")))

    # drop full_name column
    .drop("full_name")
)


In [196]:
from pyspark.sql.functions import col, trim, when

EMAIL_REGEX = (
    r'^(?=.{1,254}$)(?=.{1,64}@)'
    r'[A-Za-z0-9_%+-]+(\.[A-Za-z0-9_%+-]+)*'
    r'@'
    r'[A-Za-z0-9-]+(\.[A-Za-z0-9-]+)*\.[A-Za-z]{2,}$'
)

spark_df = spark_df.withColumn(
    "is_valid_email",
    when(col("Email").isNull(), False)
    .otherwise(trim(col("Email")).rlike(EMAIL_REGEX))
)


In [197]:
spark_df.show(truncate=False)


+---------------------+---------------+---------------------------+-------------------------------+-----------------------+--------+--------------+
|LastName             |FirstName      |Name                       |Email                          |invalid_names          |is_valid|is_valid_email|
+---------------------+---------------+---------------------------+-------------------------------+-----------------------+--------+--------------+
|as chakravarthi aaaaa|               |As Chakravarthi            |aschakrav@gmail.com            |NULL                   |true    |true          |
|raveendra   xyz      |meda           |Meda Raveendra             |.mrreddy@ramcadds.in           |  Category_Invalid_Name|false   |false         |
|kannan               |               |Kannan                     |kannan.@ican1891@gmail.com+_=  |NULL                   |true    |false         |
|ramyaa               |               |Ramyaa                     |ramyaa.doc___@gmail.com        |NULL         

In [178]:
# import pyspark.sql.functions as F

# valid_email_df = spark_df.filter(F.col("is_valid_email") == True)

# valid_email_df.show(truncate=False)


In [179]:
# import pyspark.sql.functions as F
# from pyspark.sql.window import Window

# # Window to count occurrences of each email
# email_window = Window.partitionBy("Email")

# spark_df = spark_df.withColumn(
#     "email_status",
#     F.when(
#         F.col("is_valid_email") == False,
#         "Invalid"
#     )
#     .when(
#         (F.col("is_valid_email") == True) &
#         (F.count("Email").over(email_window) > 1),
#         "Duplicate"
#     )
#     .otherwise("Unique")
# )


In [198]:
# import pyspark.sql.functions as F
# from pyspark.sql.window import Window

# # Add a temporary order column to preserve row order
# spark_df = spark_df.withColumn("original_order", F.monotonically_increasing_id())

# # Define window to count occurrences of each email
# email_window = Window.partitionBy("Email")

# # Compute email_status
# spark_df = spark_df.withColumn(
#     "email_status",
#     F.when(F.col("is_valid_email") == False, "Invalid")
#      .when((F.col("is_valid_email") == True) & (F.count("Email").over(email_window) > 1), "Duplicate")
#      .otherwise("Unique")
# )

# # Restore original order
# spark_df = spark_df.orderBy("original_order").drop("original_order")


In [202]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Add a temporary order column to preserve row order
spark_df = spark_df.withColumn("original_order", F.monotonically_increasing_id())

# Define window to count occurrences of each email
email_window = Window.partitionBy("Email")

# Compute email_status based on your rules
spark_df = spark_df.withColumn(
    "email_status",
    F.when(F.col("Email").isNull() | (F.col("Email") == ""), "Email Is Not Present")  # Email missing
     .when(F.col("is_valid_email") == False, "Invalid")                     # Email invalid
     .when(F.count("Email").over(email_window) > 1, "Duplicate")            # Email duplicated
     .otherwise("Unique")                                                    # Email valid & unique
)

# Restore original order
spark_df = spark_df.orderBy("original_order").drop("original_order")


In [201]:
spark_df.show(30,truncate=False)


+---------------------+------------------+-------------------------------------+-------------------------------+-----------------------+--------+--------------+------------+
|LastName             |FirstName         |Name                                 |Email                          |invalid_names          |is_valid|is_valid_email|email_status|
+---------------------+------------------+-------------------------------------+-------------------------------+-----------------------+--------+--------------+------------+
|as chakravarthi aaaaa|                  |As Chakravarthi                      |aschakrav@gmail.com            |NULL                   |true    |true          |Unique      |
|raveendra   xyz      |meda              |Meda Raveendra                       |.mrreddy@ramcadds.in           |  Category_Invalid_Name|false   |false         |Invalid     |
|kannan               |                  |Kannan                               |kannan.@ican1891@gmail.com+_=  |NULL              